# Simularea răspândirii gripei în București, utilizând metoda Monte Carlo

Acest proiect folosește o simulare Monte Carlo pentru a modela răspândirea gripei în București în perioada noiembrie-martie. Simularea ia în considerare probabilitatea zilnică de infectare și timpul de recuperare pentru fiecare individ din populație, împărțită pe sectoare.

In [127]:
import numpy as np
import random
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

from itertools import accumulate

## Parametrii simulării

În cadrul simulării, folosim următorii parametri:

- **Populația**: 10.000 de indivizi
- **Probabilitatea de infectare**: Valoare aleatoare între 3% și 5% pentru fiecare individ
- **Perioada de recuperare**: 7-10 zile pentru fiecare persoană infectată
- **Durata simulării**: 150 de zile (de la 1 noiembrie la 31 martie)
- **Sectoare**: Simularea este detaliată la nivel de sector, considerând 6 sectoare cu populații diferite.

Acești parametri au fost aleși pentru a reflecta dinamica unei epidemii de gripă într-un oraș mare.


In [128]:
# parametri pentru simulare
nr_populatie = 10000
prob_infectare = np.random.uniform(0.03, 0.05, nr_populatie)  # probabilitatea ca un individ sa se infecteze este intre 3-5%
nr_zile_refacere = np.zeros(nr_populatie, dtype=int)         
simulare_zile = 150                                           # nr de zile pentru care se face simularea (1 nov - 31 martie)
nr_sectoare = 6

# Simularea răspândirii gripei pe întreaga populație 

Acest cod simulează răspândirea gripei într-o populație de mărime `nr_populatie` (populația simulată pentru întreg Bucureștiul) pe o perioadă de timp definită (`simulare_zile`).

1. **Calculul Zilnic al Numărului de Infectați**:
   - `posibil_infectati`: Persoanele care nu sunt infectate (cu zile de recuperare de 0).
   - `sansa_infectare`: Valori aleatoare pentru fiecare persoană, reprezentând șansa de infectare zilnică.
   - `infectati`: Persoanele care devin infectate dacă șansa lor este mai mică decât probabilitatea de infectare.
   - **Zilele de recuperare** sunt setate între 7 și 10 zile pentru cei infectați, iar numărul zilnic de infectați este stocat în `nr_infectati_pe_zi`.
   - Zilele de recuperare sunt decrementate zilnic pentru cei infectați până când redevin susceptibili.

2. **Estimarea Probabilității de Infectare**:
   - Simulăm infecția pentru fiecare individ pe toată durata simulării.
   - **Estimarea finală**: Probabilitatea estimată de infectare este calculată ca raport între numărul de infecții și numărul total de persoane pe toată perioada simulată.

3. **Rezultate**:
   - `nr_infectati_pe_zi` conține numărul zilnic de infectați.
   - `estimare` afișează probabilitatea estimată de infectare.

Această metodă oferă o estimare a probabilității de infectare folosind simularea Monte Carlo.


In [129]:
# simulam raspandirea gripei penrtu toate zilele in populatia data
nr_infectati_pe_zi = []

for zi in range(simulare_zile):
    posibil_infectati = nr_zile_refacere == 0                           # persoanele care nu sunt inca infectate
    sansa_infectare = np.random.rand(nr_populatie) 
    infectati = posibil_infectati & (sansa_infectare < prob_infectare)  # persoane deja infectate
    nr_infectati_pe_zi.append(np.sum(infectati))

    # setam zilele de refacere pentru persoanele infectate (intre 7-10 zile)
    nr_zile_refacere[infectati] = np.random.randint(7, 11, size=np.sum(infectati))

    # decrementam zilele de refacere pentru persoanele care sunt inca infectate
    nr_zile_refacere[nr_zile_refacere > 0] -= 1


print("Nr infectati pe zi: ", nr_infectati_pe_zi)

# estimam probabilitatea de infectare cu ajutorul simularii Monte Carlo
infected_count = 0  

# rulam simularea pentru fiecare zi
for _ in range(simulare_zile):
    for i in range(nr_populatie):                # simulam pentru fiecare individ din populatie
        random_value = np.random.rand()          # generarea unei valori random 
        if random_value < prob_infectare[i]:     # daca valoarea random este mai mica decat probabilitatea de infectare, atunci individul este infectat
            infected_count += 1  

# estimare 
estimare = infected_count / (nr_populatie * simulare_zile)
print(f"Probabilitatea estimata de infectare: {estimare:.6f}")

Nr infectati pe zi:  [391, 425, 386, 343, 366, 334, 317, 290, 307, 296, 312, 311, 292, 299, 306, 303, 316, 333, 294, 301, 329, 327, 308, 301, 320, 298, 307, 312, 319, 327, 308, 298, 310, 324, 300, 290, 335, 311, 301, 293, 288, 314, 352, 283, 299, 294, 313, 310, 319, 288, 287, 340, 301, 295, 300, 312, 289, 274, 308, 311, 321, 307, 305, 314, 306, 310, 315, 288, 300, 297, 259, 279, 308, 286, 271, 298, 335, 306, 332, 313, 301, 272, 331, 313, 320, 298, 322, 347, 320, 283, 289, 325, 303, 311, 308, 317, 329, 280, 303, 282, 304, 321, 318, 315, 322, 323, 327, 321, 274, 286, 282, 321, 293, 311, 282, 330, 329, 293, 306, 325, 351, 330, 302, 293, 294, 308, 304, 320, 323, 316, 313, 304, 340, 326, 270, 322, 309, 301, 330, 295, 306, 327, 304, 318, 294, 322, 294, 304, 319, 320]
Probabilitatea estimata de infectare: 0.039653


## Rezultate și vizualizare

Rezultatele sunt reprezentate grafic folosind librăria Plotly.
Primul grafic figurează modul în care gripa se modifică în fiecare zi, începând din luna noiembrie, până în ultima zi din martie.

Acest grafic ajută la înțelegerea modului în care gripa se răspândește în oraș.


In [130]:
# plotarea rezultatelor 
zile_pe_luna = [30, 31, 31, 28, 31]
luni = ['Nov', 'Dec', 'Jan', 'Feb', 'Mar'] 
pozitie_luna = [1] + [sum(zile_pe_luna[:i]) + 1 for i in range(1, len(zile_pe_luna))]

# for i, luna in enumerate(luni):
#     print(f"1st {luna} is on day {pozitie_luna[i]}")

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(simulare_zile), 
    y=nr_infectati_pe_zi, 
    mode='lines', 
    name='Numar infectati in perioda noiembrie - martie', 
    line=dict(color='MediumPurple', width=2)))

fig.update_layout(title=dict(text='Evoluția gripei în București (noiembrie - martie)', x=0.5),
                   xaxis=dict(title='Zile', tickvals=pozitie_luna, ticktext=luni),
                   yaxis_title='Nr persoane infectate',
                   paper_bgcolor='rgba(0,0,0,0)',
                   font=dict(family='system-ui', size=14, color='white'),
                   plot_bgcolor='Gainsboro')
fig.show()

# Simularea răspândirii gripei în sectoarele Bucureștiului

Simularea modelează răspândirea gripei în București, împărțit pe 6 sectoare, pentru o perioadă de 150 de zile.

1. **Populația și Probabilitatea de Infectare**: Fiecare sector are o populație simulată și o probabilitate de infectare aleatorie între 3% și 5%.
   
2. **Zile de Recuperare și Contorizarea Infecțiilor**: Inițial, toți sunt sănătoși. `nr_zile_refacere_sectoare` contorizează zilele de recuperare, iar `nr_infectati_pe_zi_sectoare` stochează zilnic numărul de infectați pe sector.

3. **Simularea Infecțiilor**: 
   - În fiecare zi, pentru fiecare sector, persoanele susceptibile au o șansă de infectare.
   - Persoanele infectate primesc zile de recuperare între 7 și 10 zile.
   
4. **Actualizarea Stării**: Zilele de recuperare scad zilnic pentru cei infectați, iar la final se afișează numărul zilnic de infectați pe sector.

Acest model oferă o imagine a răspândirii gripei pe durata simulării în funcție de populația și probabilitatea sectorială.


In [131]:
# simularea raspandirii gripei in sectoarele Bucurestiului 
populatie_simulata_sectoare = np.array([1192, 1745, 2244, 1546, 1422, 1849])  # simularea populatiei din fiecare sector
prob_infectare_sectoare = np.random.uniform(0.03, 0.05, nr_sectoare)          # probabilitatea de infectare pentru fiecare sector

nr_zile_refacere_sectoare = [np.zeros(pop, dtype=int) for pop in populatie_simulata_sectoare] 
nr_infectati_pe_zi_sectoare = [[] for _ in range(nr_sectoare)]

for zi in range(simulare_zile):
    for s in range(nr_sectoare): # pentru fiecare sector
        posibil_infectati = nr_zile_refacere_sectoare[s] == 0 
        sansa_infectare = np.random.rand(populatie_simulata_sectoare[s])      # determinam persoanele care pot fi infectate in sectorul respectiv

        # daca sansa de infectare este mai mica decat probabilitatea de infectare, atunci persoana devine infectata
        infectati = posibil_infectati & (sansa_infectare < prob_infectare_sectoare[s])
        nr_infectati_pe_zi_sectoare[s].append(np.sum(infectati))

        # setarea zilelor de refacere pentru persoanele infectate (intre 7-10 zile)
        nr_zile_refacere_sectoare[s][infectati] = np.random.randint(7, 11, size=np.sum(infectati))

        # decrementarea zilelor de refacere pentru persoanele care sunt inca infectate
        nr_zile_refacere_sectoare[s][nr_zile_refacere_sectoare[s] > 0] -= 1

# nr de persoane infectate in fiecare sector, in fiecare zi
for s in range(nr_sectoare):
    print(f"Nr infectati pe zi in sectorul {s+1}: {nr_infectati_pe_zi_sectoare[s]}\n")


Nr infectati pe zi in sectorul 1: [46, 48, 48, 55, 36, 31, 47, 40, 26, 36, 35, 36, 32, 38, 39, 36, 34, 46, 26, 45, 38, 30, 44, 50, 37, 34, 25, 27, 38, 38, 40, 41, 32, 41, 34, 23, 35, 43, 40, 37, 42, 48, 28, 36, 36, 32, 38, 38, 29, 40, 41, 33, 31, 36, 36, 42, 42, 43, 25, 42, 38, 40, 31, 31, 31, 38, 35, 39, 44, 39, 30, 39, 38, 34, 37, 28, 43, 41, 26, 36, 31, 34, 44, 29, 31, 37, 35, 39, 47, 36, 38, 35, 31, 41, 26, 35, 35, 38, 32, 38, 34, 44, 48, 41, 35, 30, 34, 44, 34, 36, 39, 37, 26, 41, 33, 28, 44, 38, 39, 30, 42, 29, 37, 35, 44, 36, 42, 33, 30, 32, 39, 40, 29, 34, 41, 31, 41, 34, 33, 36, 29, 42, 37, 41, 26, 38, 43, 34, 33, 39]

Nr infectati pe zi in sectorul 2: [81, 83, 71, 61, 45, 68, 73, 63, 57, 59, 56, 54, 64, 66, 65, 61, 64, 65, 60, 66, 51, 57, 64, 63, 69, 61, 68, 71, 58, 71, 58, 54, 58, 69, 65, 64, 60, 55, 59, 64, 60, 48, 55, 50, 58, 62, 72, 64, 64, 62, 69, 67, 60, 63, 50, 53, 53, 60, 88, 49, 56, 53, 64, 57, 60, 61, 59, 59, 57, 58, 64, 64, 76, 67, 49, 57, 65, 64, 65, 62, 54, 58, 4

## Rezultate și Vizualizare

Acest cod estimează și vizualizează rata de infectare cu gripă în fiecare sector al Bucureștiului utilizând o hartă coropletică.

In [132]:
# nr total de infectati in fiecare sector, in fiecare zi
infectii_pe_sector_toate_zilele = [sum(nr_infectati_pe_zi_sectoare[s]) for s in range(nr_sectoare)]

# rata de infectare in fiecare sector 
infectie_rate_sectoare = [infectii / populatie_simulata_sectoare[s] for s, infectii in enumerate(infectii_pe_sector_toate_zilele)] 

# label ul pentru fiecare sector asa cum e in GeoJSON
label_sector = ['Sector 1', 'Sector 2', 'Sector 3', 'Sector 4', 'Sector 5', 'Sector 6']

# dataframe cu rata de infectare pentru fiecare sector
data_rate = pd.DataFrame({'Sector': label_sector, 'Infection Rate': infectie_rate_sectoare})
print(data_rate)

# incarcarea fisierului GeoJSON 
geojson_fisier = '/Users/alexandrazamfirescu/Flu_Spread_Simulation/BucurestiSectoare.geo.json'

with open(geojson_fisier) as f:
    geojson = json.load(f)

# reprezentare harta coropletica 
fig = px.choropleth(
    data_frame=data_rate,
    geojson=geojson_fisier,
    locations='Sector',
    featureidkey='properties.tags.name', 
    color='Infection Rate',
    color_continuous_scale='Turbo',
    labels={'Infection Rate': 'Infection Rate'},
    projection='mercator'
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title=dict(text='Rata de infectare cu gripa in fiecare sector', x=0.5),
    margin={"r":0, "t":40, "l":0, "b":0}
)

fig.show()


     Sector  Infection Rate
0  Sector 1        4.605705
1  Sector 2        5.236103
2  Sector 3        3.769608
3  Sector 4        4.417853
4  Sector 5        4.637131
5  Sector 6        4.292591


## Concluzii

Această simulare oferă o estimare a răspândirii gripei în București, ținând cont de factori specifici fiecărui sector. Deși modelul este o simplificare, acesta poate fi extins pentru a include alți factori, cum ar fi mobilitatea între sectoare sau intervențiile de sănătate publică. Rezultatele pot fi utilizate pentru a evalua riscurile de epidemie în funcție de parametrii de infectare și recuperare.

Prin utilizarea metodei Monte Carlo, simularea captează variabilitatea zilnică a răspândirii gripei și poate fi utilizată pentru predicții și analize epidemiologice în viitor.
